In [7]:
import torch
import pandas as pd
from PIL import Image
import gradio as gr
from datasets import Dataset
from qwen_vl_utils import process_vision_info  # 이미지/비디오 전처리를 위한 함수
from transformers import AutoProcessor, AutoModelForVision2Seq
from sklearn.model_selection import train_test_split

# ----------------------------
# 1. 모델 및 프로세서 로드
# ----------------------------

# Qwen2-VL-7B-Instruct 모델과 프로세서를 불러옵니다.
model_id = "Qwen/Qwen2-VL-7B-Instruct"
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",          # 가능한 GPU에 할당
    torch_dtype=torch.float16   # 메모리 효율성을 위한 float16 사용
)
processor = AutoProcessor.from_pretrained(model_id)

# 로라 어댑터(파인튜닝된 가중치)를 로드하고 모델에 적용합니다.
adapter_path1 = "./qwen2-7b-instruct-harmful-detector-2000/checkpoint-32"
model.load_adapter(adapter_path1, adapter_name="adapter1")
model.set_adapter("adapter1")

# ----------------------------
# 2. 프롬프트 및 메시지 형식 정의
# ----------------------------

# 시스템 메시지: 모델의 역할을 정의합니다.
system_message = "당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다."

# 프롬프트 템플릿: 모델이 유해성 여부를 판단할 때 참고할 기준 및 출력 형식을 지정합니다.
prompt = """입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 '1' 또는 '0'으로 출력하세요."""

system_message = "당신은 이미지와 텍스트의 '유해성(harmfulness)'을 판별하는 분류 모델입니다."

# 프롬프트 템플릿: 모델이 유해성 여부를 판단할 때 참고할 기준 및 출력 형식을 지정합니다.
prompt = """입력으로 이미지 또는 ###텍스트가 주어지면, 해당 콘텐츠가 유해한지 여부를 "1" 또는 "0"으로 출력하세요.
- 출력은 반드시 숫자 하나만(문자열 없이) 출력합니다.
  - "1" : 유해(혐오·차별·폭력·선정성·기타 해악 가능성이 있다고 판단되는 경우)
  - "0" : 비유해(문제 없다고 판단되는 경우)
유해성(harmfulness)을 판단할 때는 아래 항목을 포함해 검토하세요:
1. **혐오 표현 (Hate Speech)**
   - 인종, 성별, 종교, 성적 지향, 장애 등을 근거로 특정 집단/개인을 비하·모욕·차별하는 언어/이미지
   - 명백한 증오, 차별, 폭력 조장 등
2. **직접적인 폭력·잔혹 행위**
   - 극단적인 폭력 묘사(살해·자해·테러 등)
   - 노골적으로 잔혹하거나 충격적인 내용
3. **노골적 성적 콘텐츠**
   - 명백히 선정적·음란한 묘사, 아동 성착취 등 불법적이거나 심각한 윤리적 문제가 있는 경우
4. **사생활 침해, 불법 행위 조장**
   - 명백한 사생활 침해, 범죄 또는 불법 행위를 조장하는 내용
5. **기타 위험 요소**
   - 위 항목 외에도, 사회적으로 유해하거나 심각한 해악을 유발할 수 있는 이미지·텍스트
만약 콘텐츠가 위 기준 중 하나라도 충족하여 "유해"하다고 판단되면 "1"을,
해당되지 않으면 "0"을 출력하십시오.
출력은 오직 숫자 하나(1 또는 0)만 반환하고, 어떠한 추가 문구나 설명도 첨부하지 마십시오.

###텍스트 :{text}"""

# format_data 함수: CSV의 한 샘플 정보를 받아서 모델에 입력할 대화 메시지 형식으로 변환합니다.
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",  # 시스템 역할 메시지
                "content": [{"type": "text", "text": system_message}],
            },
            {
                "role": "user",  # 사용자 역할 메시지: 텍스트 프롬프트와 이미지 정보를 포함
                "content": [
                    {
                        "type": "text",
                        "text": prompt.format(text=sample["translated"]),
                    },
                    {
                        "type": "image",
                        "image": sample["file_path"],  # 이미지 파일 경로 (필요에 따라 PIL.Image 객체로 변경 가능)
                    }
                ],
            },
            {
                "role": "assistant",  # 어시스턴트 역할 메시지: 실제 정답(ground truth) 포함
                "content": [{"type": "text", "text": sample["is_hate"]}],
            },
        ]
    }

# ----------------------------
# 3. 모델 추론 함수 정의
# ----------------------------

# generate_description: 모델 추론을 수행하여 생성된 텍스트를 반환합니다.
def generate_description(messages, model, processor):
    # 채팅 템플릿을 적용하여 텍스트로 변환합니다.
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    # 이미지 및 비디오 입력을 전처리합니다.
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)

    # 모델 추론: 새로운 토큰을 생성합니다.
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128,
        top_p=0.95,
        do_sample=True,
        temperature=0.1
    )

    # 원본 입력 토큰 수 만큼 잘라내어 생성된 부분만 추출합니다.
    generated_ids_trimmed = [
        out_ids[len(in_ids):]
        for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    # 토큰을 텍스트로 디코딩합니다.
    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return output_text[0]

# ----------------------------
# 4. 데이터셋 로드 및 전처리
# ----------------------------

# CSV 파일에서 데이터를 로드하고, 필요한 전처리(숫자형 변환, 샘플 셔플, 인덱스 리셋 등)를 수행합니다.
df = pd.read_csv('./data/final_df.csv')
df["is_hate"] = df["is_hate"].astype(int)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 데이터를 train과 test로 분할합니다.
train_dataset, test_dataset = train_test_split(df, test_size=0.1, random_state=42)
# test_dataset의 인덱스를 0부터 순차적으로 재설정합니다.
test_dataset = test_dataset.reset_index(drop=True)

# ----------------------------
# 5. 데이터 샘플 로드 함수
# ----------------------------
# 선택한 인덱스의 샘플 데이터를 불러와 이미지와 텍스트를 반환합니다.
def load_sample(sample_index):
    idx = int(sample_index)
    row = test_dataset.iloc[idx]
    image_path = row["file_path"]
    text = row["translated"]
    try:
        image = Image.open(image_path)
    except Exception as e:
        image = None  # 이미지 로드 실패 시 None 반환
    return image, text

# ----------------------------
# 6. 모델 추론 및 결과 출력 함수
# ----------------------------

# 선택한 샘플에 대해 모델 추론을 수행하고, 예측 결과와 실제 정답을 함께 반환합니다.
def run_inference(sample_index, user_text):
    idx = int(sample_index)
    row = test_dataset.iloc[idx]

    # CSV 샘플 데이터를 대화 형식 메시지로 변환 (실제 정답 포함)
    messages = format_data(row)["messages"]
    # 모델을 통해 예측 텍스트를 생성합니다.
    prediction_text = generate_description(messages, model, processor)

    # 모델 예측 결과를 숫자로 변환한 후 해석합니다.
    try:
        prediction_int = int(prediction_text.strip())
    except ValueError:
        predicted_label = f"예측 결과를 숫자로 변환하지 못했습니다: {prediction_text}"
    else:
        if prediction_int == 1:
            predicted_label = "예측: 혐오 데이터입니다."
        else:
            predicted_label = "예측: 비혐오 데이터입니다."

    # CSV에 기록된 실제 정답을 확인합니다.
    actual_int = int(row["is_hate"])
    if actual_int == 1:
        actual_label = "실제 정답: 혐오 데이터입니다."
    else:
        actual_label = "실제 정답: 비혐오 데이터입니다."

    # 예측 결과와 실제 정답을 함께 반환합니다.
    return f"{predicted_label}\n{actual_label}"

# ----------------------------
# 7. Gradio UI 설정
# ----------------------------

# 사용자 인터페이스(UI)를 Gradio Blocks로 구성합니다.
custom_css = """
<style>
#title {
    text-align: center;
    font-size: 2em;
    font-weight: bold;
    margin: 20px 0 10px 0;
}
#subtitle {
    text-align: center;
    font-size: 1.1em;
    color: #666;
    margin-bottom: 30px;
}
.gr-button {
    background-color: #4CAF50 !important;
    color: white !important;
    border: none !important;
}
.gr-textbox textarea {
    min-height: 120px !important;
}
</style>
"""

with gr.Blocks(custom_css) as demo:
    # 제목 및 부제목 표시
    gr.Markdown("<div id='title'>이미지 & 텍스트 입력을 통한 모델 예측 데모</div>")
    gr.Markdown("<p id='subtitle'>아래에서 샘플을 선택하거나 직접 텍스트를 입력하고 예측을 실행해보세요.</p>")

    # 샘플 인덱스 선택 및 샘플 불러오기 버튼
    with gr.Row():
        sample_index = gr.Dropdown(
            choices=[str(i) for i in test_dataset.index],
            label="샘플 인덱스 선택",
            value="0",
            interactive=True
        )
        update_button = gr.Button("샘플 불러오기")

    # 이미지 및 텍스트 입력/출력 영역 구성
    with gr.Row():
        with gr.Column():
            image_output = gr.Image(label="선택된 이미지", elem_id="image_display")
        with gr.Column():
            text_input = gr.Textbox(label="텍스트 입력", placeholder="분석할 텍스트를 입력하세요.")
            predict_button = gr.Button("예측 실행", variant="primary")
            result_output = gr.Textbox(label="예측 결과", placeholder="결과가 여기에 표시됩니다.")

    # 버튼 클릭 시 함수 연결
    update_button.click(fn=load_sample, inputs=sample_index, outputs=[image_output, text_input])
    predict_button.click(fn=run_inference, inputs=[sample_index, text_input], outputs=result_output)

# Gradio 앱 실행 (share=True 옵션으로 외부 접근 가능, debug=True로 디버깅 정보 표시)
demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://1591d381997255d4ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1591d381997255d4ea.gradio.live
